### Sensitive analysis:


In [1]:
from sast.sast import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd


/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/mass_ts/_mass_ts.py:17: UserWarning: GPU support will not work. You must pip install mass-ts[gpu].
  warnings.warn(


In [15]:
ds_sens = tsc_dataset_names.univariate_equal_length
#ds_sens = ['ACSF1', 'Adiac', 'ArrowHead', 'Beef', 'BeetleFly', 'BirdChicken', 'BME', 'Car', 'CBF', 'Chinatown', 'ChlorineConcentration', 'CinCECGTorso', 'Coffee']
ds_sens = ['Chinatown','BirdChicken','Coffee' ]
max_ds=len(ds_sens) #exploring dataset in UEA & UCR Time Series Classification Repository
#print(ds_sens)
print(ds_sens)


['Chinatown', 'BirdChicken', 'Coffee']


In [16]:
#ds='Chinatown' # Chosing a dataset from # Number of classes to consider
runs = 1
range_total = [1,10]
#list
df_result = {}
list_score = []
list_time = []
list_dataset = []
list_hyperparameter = []
list_method = []
list_rpoint = []
list_nb_per_class = []

for ds in ds_sens:
    print("ds="+ds)
    X_train, y_train = load_UCR_UEA_dataset(name=ds, extract_path='data', split="train", return_type="numpy2d")
    X_test, y_test = load_UCR_UEA_dataset(name=ds, extract_path='data', split="test", return_type="numpy2d")
    for i in range(runs):
        """
        start = time.time()
        rocket= RocketClassifier(num_kernels=10_000)
        rocket.fit(X_train,y_train)
        end = time.time()
        list_score.append(rocket.score(X_test,y_test))
        list_time.append(end-start)
        list_dataset.append(ds)
        list_hyperparameter.append("num_kernels=10_000")
        list_rpoint.append(str(0))
        list_nb_per_class.append(str(0))
        list_method.append("Rocket")
        """
        for p in range_total:
            for g in range_total:
                if p ==0:
                    p=1
                if g ==0:
                    g=1
                start = time.time()
                random_state = None
                rsast_ridge = RSAST(n_random_points=p,nb_inst_per_class=g, classifier=RidgeClassifierCV())
                rsast_ridge.fit(X_train, y_train)
                end = time.time()
                list_score.append(rsast_ridge.score(X_test,y_test))
                list_time.append(end-start)
                list_dataset.append(ds)
                list_hyperparameter.append("n_random_points="+str(p)+" nb_inst_per_class="+str(g))
                list_rpoint.append(str(p))
                list_nb_per_class.append(str(g))
                list_method.append("Rsast")
                print("n_random_points="+str(p)+" nb_inst_per_class="+str(g))

df_result['accuracy']=list_score
df_result['time']=list_time
df_result['dataset_name']=list_dataset
df_result['classifier_name']=list_hyperparameter
df_result['rpoint']=list_rpoint
df_result['nb_per_class']=list_nb_per_class
df_result['method']=list_method

ds=Chinatown
n_random_points=1 nb_inst_per_class=1
n_random_points=1 nb_inst_per_class=10
n_random_points=10 nb_inst_per_class=1
n_random_points=10 nb_inst_per_class=10
ds=BirdChicken
n_random_points=1 nb_inst_per_class=1
n_random_points=1 nb_inst_per_class=10
n_random_points=10 nb_inst_per_class=1
n_random_points=10 nb_inst_per_class=10
ds=Coffee
n_random_points=1 nb_inst_per_class=1
n_random_points=1 nb_inst_per_class=10
n_random_points=10 nb_inst_per_class=1
n_random_points=10 nb_inst_per_class=10


In [17]:
df_result=pd.DataFrame(df_result)
#df_result.to_csv("df_tunning_hyper.csv") 
df_result

,accuracy,time,dataset_name,classifier_name,rpoint,nb_per_class,method
0,0.775510,0.012134,Chinatown,n_random_points=1 nb_inst_per_class=1,1,1,Rsast
1,0.897959,0.044127,Chinatown,n_random_points=1 nb_inst_per_class=10,1,10,Rsast
2,0.935860,0.065233,Chinatown,n_random_points=10 nb_inst_per_class=1,10,1,Rsast
3,0.985423,0.138541,Chinatown,n_random_points=10 nb_inst_per_class=10,10,10,Rsast
4,0.800000,3.359411,BirdChicken,n_random_points=1 nb_inst_per_class=1,1,1,Rsast
5,0.800000,31.225679,BirdChicken,n_random_points=1 nb_inst_per_class=10,1,10,Rsast
6,0.650000,7.565690,BirdChicken,n_random_points=10 nb_inst_per_class=1,10,1,Rsast
7,0.850000,69.775809,BirdChicken,n_random_points=10 nb_inst_per_class=10,10,10,Rsast
8,0.964286,0.889015,Coffee,n_random_points=1 nb_inst_per_class=1,1,1,Rsast
9,1.000000,7.286459,Coffee,n_random_points=1 nb_inst_per_class=10,1,10,Rsast


In [22]:
# create a pivot table
pivot_table = pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')

print(pivot_table)
pivot_table.to_csv("df_tunning_hyper.csv") 

                                                      accuracy
classifier_name                         dataset_name          
n_random_points=1 nb_inst_per_class=1   BirdChicken   0.800000
                                        Chinatown     0.775510
                                        Coffee        0.964286
n_random_points=1 nb_inst_per_class=10  BirdChicken   0.800000
                                        Chinatown     0.897959
                                        Coffee        1.000000
n_random_points=10 nb_inst_per_class=1  BirdChicken   0.650000
                                        Chinatown     0.935860
                                        Coffee        1.000000
n_random_points=10 nb_inst_per_class=10 BirdChicken   0.850000
                                        Chinatown     0.985423
                                        Coffee        1.000000
